In [69]:
"""here this is the code of the face detection"""

import cv2

# Method to draw boundary around the detected feature
def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text):
    # Converting image to gray-scale
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # detecting features in gray-scale image, returns coordinates, width and height of features
    features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)
    coords = []
    # drawing rectangle around the feature and labeling it
    for (x, y, w, h) in features:
        cv2.rectangle(img, (x,y), (x+w, y+h), color, 2)
        cv2.putText(img, text, (x, y-4), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
        coords = [x, y, w, h]
    return coords


# Method to detect the features
def detect(img, faceCascade, eyeCascade, noseCascade, mouthCascade):
    color = {"blue":(255,0,0), "red":(0,0,255), "green":(0,255,0), "white":(255,255,255)}
    coords = draw_boundary(img, faceCascade, 1.1, 10, color['blue'], "Face")
    # If feature is detected, the draw_boundary method will return the x,y coordinates and width and height of rectangle else the length of coords will be 0
    if len(coords)==4:
        # Updating region of interest by cropping image
        roi_img = img[coords[1]:coords[1]+coords[3], coords[0]:coords[0]+coords[2]]
        # Passing roi, classifier, scaling factor, Minimum neighbours, color, label text
        coords = draw_boundary(roi_img, eyeCascade, 1.1, 5, color['red'], "Eye")
        coords = draw_boundary(roi_img, noseCascade, 1.1, 5, color['green'], "Nose")
        coords = draw_boundary(roi_img, mouthCascade, 1.1, 10, color['white'], "Mouth")
    return img


# Loading classifiers
faceCascade = cv2.CascadeClassifier('C:/Users/Rajneesh/Downloads/Compressed/FaceDetection-master/frontalface.xml')
eyesCascade = cv2.CascadeClassifier('C:/Users/Rajneesh/Desktop/eyes.xml')
noseCascade = cv2.CascadeClassifier('C:/Users/Rajneesh/Downloads/Compressed/Nose25x15.1/Nariz.xml')
mouthCascade = cv2.CascadeClassifier('C:/Users/Rajneesh/Downloads/Compressed/Mouth25x15.1/Mouth.xml')

video_capture = cv2.VideoCapture(0)

frame_width = int(video_capture.get(9))
frame_height = int(video_capture.get(16))

while True:
    # Reading image from video stream
    _, img = video_capture.read()
    # Call method we defined above
    img = detect(img, faceCascade, eyesCascade, noseCascade, mouthCascade)
    # Writing processed image in a new window
    cv2.imshow("face detection", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# releasing web-cam
video_capture.release()
# Destroying output window
cv2.destroyAllWindows()


# Face Detection and Generating Dataset

In [12]:
import cv2

def generate_dataset(img,id,img_id):
    cv2.imwrite("D:/images/user."+str(id)+"."+str(img_id)+".jpg",img)

# Method to draw boundary around the detected feature
def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text):
    # Converting image to gray-scale
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # detecting features in gray-scale image, returns coordinates, width and height of features
    features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)
    coords = []
    # drawing rectangle around the feature and labeling it
    for (x, y, w, h) in features:
        cv2.rectangle(img, (x,y), (x+w, y+h), color, 2)
       	cv2.putText(img,text, (x, y-4), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
        coords = [x, y, w, h]
    return coords


# Method to detect the features
def detect(img, faceCascade,img_id):
    color = {"blue":(255,0,0), "red":(0,0,255), "green":(0,255,0), "white":(255,255,255)}
    coords = draw_boundary(img, faceCascade, 1.1, 10, color['white'], "New_person")
    # If feature is detected, the draw_boundary method will return the x,y coordinates and width and height of rectangle else the length of coords will be 0
    if len(coords)==4:
        roi_img = img[coords[1]:coords[1]+coords[3], coords[0]:coords[0]+coords[2]]
        user_id =4
        generate_dataset(roi_img,user_id,img_id)
    return img

# Loading face classifier
faceCascade = cv2.CascadeClassifier('C:/Users/Rajneesh/Downloads/Compressed/FaceDetection-master/frontalface.xml')
video_capture = cv2.VideoCapture(0)
img_id=0 

while True:
    # Reading image from video stream
    _, img = video_capture.read()
    # Call method we defined above
    img = detect(img, faceCascade,img_id)
    cv2.imshow("face detection", img)
    img_id+=1
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()


# CLASSIFIER FILE

In [3]:
import numpy as np 
from PIL import Image
import os 
import cv2
data='D:/images'
def train_classifier(data):
    #creating a list that contains all the files (photos)
    path = [os.path.join(data,f) for f in os.listdir(data)]
    faces = []
    ids = []

    for image in path:
        img = Image.open(image).convert('L')
        imageNp=np.array(img, 'uint8')
        id = int(os.path.split(image)[1].split(".")[1])
        faces.append(imageNp)
        ids.append(id)

    ids = np.array(ids)
    clf=cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces,ids)
    clf.write("Classifier.yml")
train_classifier(data)


# Recognizer File

In [5]:
import cv2

def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf):
    # Converting image to gray-scale
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # detecting features in gray-scale image, returns coordinates, width and height of features
    features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)
    coords = []

    # drawing rectangle around the feature and labeling it
    for (x, y, w, h) in features:
        cv2.rectangle(img, (x,y), (x+w, y+h), color, 2)
        # Predicting the id of the user
        id, _ = clf.predict(gray_img[y:y+h, x:x+w])
        # Check for id of user and label the rectangle accordingly
        if id == 1:
            cv2.putText(img, "Rajneesh", (x, y-4), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
        if id == 2: 
            cv2.putText(img,"Virat_Kohli", (x,y-4),cv2.FONT_HERSHEY_SIMPLEX,0.8,color,1,cv2.LINE_AA)
        if id == 3: 
            cv2.putText(img,"Shahrukh_Khan", (x,y-4),cv2.FONT_HERSHEY_SIMPLEX,0.8,color,1,cv2.LINE_AA)
        if id == 4: 
            cv2.putText(img,"Akshay_kumar", (x,y-4),cv2.FONT_HERSHEY_SIMPLEX,0.8,color,1,cv2.LINE_AA)
        if id == 5: 
            cv2.putText(img,"Captain_America", (x,y-4),cv2.FONT_HERSHEY_SIMPLEX,0.8,color,1,cv2.LINE_AA)
       
        coords = [x, y, w, h]
    return coords


# Method to recognize the person
def recognize(img, clf, faceCascade):
    color = {"black":(0,0,0),"blue": (255, 0, 0), "red": (0, 0, 255), "green": (0, 255, 0), "white": (255, 255, 255)}
    coords = draw_boundary(img, faceCascade, 1.1, 10, color["green"], "Face", clf)
    return img


# Loading classifier
faceCascade = cv2.CascadeClassifier('C:/Users/Rajneesh/Downloads/Compressed/FaceDetection-master/frontalface.xml')

# Loading custom classifier to recognize
clf = cv2.face.LBPHFaceRecognizer_create()
clf.read("C:/Users/Rajneesh/Documents/jupyter/Classifier.yml")

# Capturing real time video stream. 0 for built-in web-cams, 0 or -1 for external web-cams
video_capture = cv2.VideoCapture(0)

while True:
    # Reading image from video stream
    _, img = video_capture.read()
    # Call method we defined above
    img = recognize(img, clf, faceCascade)
    # Writing processed image in a new window
    cv2.imshow("face detection", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# releasing web-cam
video_capture.release()
# Destroying output window
cv2.destroyAllWindows()